# This script generates Data representable by the netherlands it is syntethic data that mirrors the netherlands tax payers and its sectors based on the open source data we could find out there

In [7]:
import random
import csv

# =========================================================
# CONFIG – GLOBAL SETTINGS (PANEL)
# =========================================================

N_FIRMS = 30000  # number of unique firms

YEARS = list(range(2005, 2025))  # 2005–2024

GROOTTEKLASSEN = ["micro", "klein", "midden"]
GROOTTE_PROBS = [0.82, 0.14, 0.04]

SECTORS = [
    "Detailhandel", "Horeca", "Bouw", "Landbouw & Visserij", "Industrie",
    "Vervoer & Logistiek", "Informatie & ICT", "Financiële dienstverlening",
    "Zakelijke diensten", "Zorg & Welzijn", "Onderwijs & Overheid",
    "Creatieve industrie & Media", "Persoonlijke dienstverlening",
]

SECTOR_PROBS = [
    0.16, 0.07, 0.11, 0.04, 0.06,
    0.06, 0.07, 0.04, 0.18, 0.08,
    0.03, 0.05, 0.05,
]

BUSINESS_TYPES = {
    "Horeca": ["Kebabzaak","Snackbar","Grillroom","Restaurant","Café-bar","Lunchroom","IJssalon"],
    "Detailhandel": ["Supermarkt","Kledingwinkel","Elektronicazaak","Boekenwinkel","Drogisterij","Speciaalzaak delicatessen","Webshop"],
    "Bouw": ["Schildersbedrijf","Aannemersbedrijf","Tegelzetter","Loodgietersbedrijf","Elektra-installatiebedrijf"],
    "Landbouw & Visserij": ["Melkveebedrijf","Akkerbouwbedrijf","Tuinbouwbedrijf","Varkenshouderij","Visserijbedrijf"],
    "Industrie": ["Metaalbewerkingsbedrijf","Voedingsmiddelenfabriek","Textielfabriek","Machinebouw"],
    "Vervoer & Logistiek": ["Transportbedrijf","Koeriersdienst","Distributiecentrum","Taxibedrijf"],
    "Informatie & ICT": ["Softwarebedrijf","IT-consultancy","Hostingprovider","Managed services provider","Data-analysebureau"],
    "Financiële dienstverlening": ["Assurantiekantoor","Financieel adviesbureau","Vermogensbeheerder","FinTech-startup"],
    "Zakelijke diensten": ["Administratiekantoor","Accountantskantoor","Marketingbureau","HR-adviesbureau","Organisatieadviesbureau","Detacheringbureau"],
    "Zorg & Welzijn": ["Huisartsenpraktijk","Fysiotherapiepraktijk","Thuiszorgorganisatie","Psychologenpraktijk","Tandartspraktijk"],
    "Onderwijs & Overheid": ["Particulier opleidingsinstituut","Taalinstituut","Adviesorgaan overheid","Onderwijsadviesbureau"],
    "Creatieve industrie & Media": ["Reclamebureau","Fotostudio","Videoproductiebedrijf","Ontwerpstudio","Game-studio"],
    "Persoonlijke dienstverlening": ["Nagelsalon","Kapsalon","Schoonheidssalon","Barbershop","Tattoo- en piercingstudio"],
}

RECHTSVORMEN = ["eenmanszaak","BV","VOF","maatschap"]
RECHTSVORM_PROBS = [0.50, 0.30, 0.15, 0.05]

REGIOS = ["Randstad","Noord","Oost","Zuid","Midden"]
REGIO_PROBS = [0.4,0.12,0.18,0.18,0.12]

PLAATSEN = ["Staphorst","Urk","Grootstedelijke gemeente","Middelgrote gemeente","Plattelandsgemeente"]
PLAATS_PROBS = [0.01,0.01,0.38,0.30,0.30]

DIGI_LEVELS = ["laag","middel","hoog"]
DIGI_PROBS_OVERALL = [0.35,0.40,0.25]

DIGI_SECTOR_BIAS = {
    "Informatie & ICT": (0.1,0.3,0.6),
    "Zakelijke diensten": (0.2,0.45,0.35),
    "Detailhandel": (0.35,0.40,0.25),
    "Horeca": (0.45,0.40,0.15),
    "Bouw": (0.45,0.40,0.15),
    "Persoonlijke dienstverlening": (0.50,0.35,0.15),
}

SECTOR_MARGIN_RANGES = {
    "Detailhandel": (0.03,0.10), "Horeca": (0.02,0.08), "Bouw": (0.04,0.12),
    "Landbouw & Visserij": (0.02,0.08), "Industrie": (0.05,0.15),
    "Vervoer & Logistiek": (0.03,0.10), "Informatie & ICT": (0.10,0.25),
    "Financiële dienstverlening": (0.15,0.30),"Zakelijke diensten": (0.10,0.22),
    "Zorg & Welzijn": (0.04,0.12),"Onderwijs & Overheid": (0.03,0.08),
    "Creatieve industrie & Media": (0.05,0.18),"Persoonlijke dienstverlening": (0.05,0.20),
}

HIGHER_RISK_SECTORS = {
    "Horeca","Detailhandel","Bouw","Vervoer & Logistiek","Persoonlijke dienstverlening"
}

HIGHER_RISK_BUSINESSES = {
    "Kebabzaak","Snackbar","Grillroom","Nagelsalon","Kapsalon",
    "Barbershop","Schoonheidssalon","Tattoo- en piercingstudio","Taxibedrijf"
}

PATTERN_TYPES = ["normal","stuck_in_cash"]
PATTERN_PROBS = [0.97,0.03]

P_ANOMALY = 0.01


# =========================================================
# HELPERS
# =========================================================

def weighted_choice(options, weights):
    return random.choices(options, weights=weights, k=1)[0]

def sample_owner_birth():
    r=random.random()
    if r<0.20: age_2005=random.randint(18,30)
    elif r<0.55: age_2005=random.randint(31,45)
    elif r<0.90: age_2005=random.randint(46,65)
    else: age_2005=random.randint(66,80)
    birth_year=2005-age_2005
    month=random.randint(1,12)
    day=random.randint(1,28)
    return birth_year, f"{birth_year}-{month:02d}-{day:02d}"

def owner_age_in_year(birth_year,year):
    return year-birth_year

def sample_firm_lifecycle():
    r=random.random()
    if r<0.40: start=random.randint(2005,2008)
    elif r<0.75: start=random.randint(2009,2016)
    else: start=random.randint(2017,2024)
    r2=random.random()
    if r2<0.60: end=2024
    else: end=min(2024,start+random.randint(1,8))
    return start,end

def sample_jaaromzet(size):
    if size=="micro": return random.randint(80_000,400_000)
    if size=="klein": return random.randint(400_000,2_500_000)
    return random.randint(2_500_000,20_000_000)

def sample_workers(size):
    if size=="micro": return random.randint(1,9)
    if size=="klein": return random.randint(10,49)
    return random.randint(50,249)

def sample_profit(turnover,sector):
    low,high=SECTOR_MARGIN_RANGES.get(sector,(0.05,0.15))
    return int(turnover*random.uniform(low,high))

def sample_branches(sec):
    arr=BUSINESS_TYPES.get(sec)
    return random.choice(arr) if arr else sec

def sample_regio():
    return weighted_choice(REGIOS,REGIO_PROBS)

def sample_plaats():
    p=weighted_choice(PLAATSEN,PLAATS_PROBS)
    return p, (p in {"Staphorst","Urk"})

def sample_digitalisation(sector,year,pattern):
    if pattern=="stuck_in_cash":
        return weighted_choice(DIGI_LEVELS,[0.65,0.30,0.05])
    if sector in DIGI_SECTOR_BIAS:
        low,mid,high = DIGI_SECTOR_BIAS[sector]
    else:
        low,mid,high = DIGI_PROBS_OVERALL
    trend=(year-2005)/19
    low=max(0.05,low-0.5*0.3*trend)
    total=low+mid+high
    mid+=0.6*(total-low-mid-high)
    high+=0.4*(total-low-mid-high)
    return weighted_choice(DIGI_LEVELS,[low,mid,high])

def sample_fiscal(size,digi):
    base=0.4
    if size=="midden": base+=0.3
    elif size=="klein": base+=0.15
    if digi=="hoog": base+=0.1
    return "ja" if random.random()<min(max(base,0.05),0.95) else "nee"

def sample_cash(sector,branche,year,digi,age,pattern):
    if branche in HIGHER_RISK_BUSINESSES: p=0.9
    elif sector in HIGHER_RISK_SECTORS: p=0.6
    else: p=0.1
    if year>=2016: p-=0.15
    if year>=2020: p-=0.10
    if digi=="hoog": p-=0.15
    if age>=67: p+=0.20
    if pattern=="stuck_in_cash": p=max(p,0.75)
    return "ja" if random.random()<max(min(p,0.95),0.02) else "nee"

def sample_prior_control():
    r=random.random()
    if r<0.75: return "geen"
    if r<0.93: return "administratief"
    return "boekenonderzoek"

def sample_voluntary(year,sector,branche,digi,advisor,cash,bench,control,age,pattern):
    score=0.5
    if year>=2015: score+=0.05
    if year>=2020: score+=0.05
    if digi=="hoog": score+=0.10
    if advisor=="ja": score+=0.15
    if bench: score+=0.10
    if control=="boekenonderzoek": score+=0.10
    if cash=="ja": score-=0.10
    if age>=67: score+=0.05
    if pattern=="stuck_in_cash": score-=0.05
    score=max(0,min(1,score))
    if score<0.4: lvl="laag"
    elif score<0.7: lvl="middel"
    else: lvl="hoog"
    return score,lvl

def taxation(winst,rechtsvorm):
    if winst<=0: return 0
    if rechtsvorm in {"eenmanszaak","VOF","maatschap"}:
        return int(winst*0.30)
    if winst<=200_000: return int(winst*0.19)
    return int(200_000*0.19 + (winst-200_000)*0.25)

def risk_and_filing(sector,size,branche,digi,advisor,cash,control,vol_score):
    r=0.20
    if sector in HIGHER_RISK_SECTORS: r+=0.20
    if branche in HIGHER_RISK_BUSINESSES: r+=0.10
    if size=="micro": r+=0.10
    if cash=="ja": r+=0.10
    if digi=="hoog": r-=0.10
    if advisor=="ja": r-=0.10
    if control=="boekenonderzoek": r-=0.10
    r-=0.20*(vol_score-0.5)
    r=max(0.05,min(0.90,r))
    hist=r
    pg=r*0.4
    pk=r*0.6
    rg=random.random()
    if rg<pg: correct="foutief_groot"
    elif rg<pg+pk: correct="foutief_klein"
    else: correct="correct"
    rt=random.random()
    if rt<r*0.25: t="geen_aangifte"
    elif rt<r*0.7: t="te_laat"
    else: t="op_tijd"
    rb=random.random()
    if correct=="foutief_groot" or t=="geen_aangifte":
        if rb<0.45: pay="niet_betaald"
        elif rb<0.85: pay="gedeeltelijk_betaald"
        else: pay="volledig_betaald"
    elif correct=="foutief_klein":
        if rb<0.12: pay="niet_betaald"
        elif rb<0.40: pay="gedeeltelijk_betaald"
        else: pay="volledig_betaald"
    else:
        pay="volledig_betaald" if rb>=0.04 else "gedeeltelijk_betaald"
    score=hist
    if correct=="foutief_groot": score+=0.30
    elif correct=="foutief_klein": score+=0.15
    if pay=="niet_betaald": score+=0.30
    elif pay=="gedeeltelijk_betaald": score+=0.15
    if t=="geen_aangifte": score+=0.25
    elif t=="te_laat": score+=0.10
    score=max(0,min(1,score))
    if score<0.35: risk_cat="laag"
    elif score<0.7: risk_cat="middel"
    else: risk_cat="hoog"
    return pay, correct, t, risk_cat, round(hist,3)


# =========================================================
# PANEL DATA GENERATION
# =========================================================

def generate_panel(n):
    records=[]
    firms=[]
    for i in range(1,n+1):
        bid=f"MKB{str(i).zfill(5)}"
        pattern=weighted_choice(PATTERN_TYPES,PATTERN_PROBS)
        by,bdate=sample_owner_birth()
        start,end=sample_firm_lifecycle()
        size=weighted_choice(GROOTTEKLASSEN,GROOTTE_PROBS)
        sector=weighted_choice(SECTORS,SECTOR_PROBS)
        branche=sample_branches(sector)
        rechtsvorm=weighted_choice(RECHTSVORMEN,RECHTSVORM_PROBS)
        regio=sample_regio()
        plaats,bench=sample_plaats()
        base_workers=sample_workers(size)
        firms.append({
            "bid":bid,"pattern":pattern,
            "birth_year":by,"birthdate":bdate,
            "start":start,"end":end,
            "size":size,"sector":sector,"branch":branche,
            "rechtsvorm":rechtsvorm,"regio":regio,"plaats":plaats,
            "bench":bench,"base_workers":base_workers
        })
    for f in firms:
        for year in YEARS:
            if not(f["start"]<=year<=f["end"]): continue
            age=owner_age_in_year(f["birth_year"],year)
            workers=max(1,f["base_workers"]+random.randint(-2,3))
            digi=sample_digitalisation(f["sector"],year,f["pattern"])
            advisor=sample_fiscal(f["size"],digi)
            control=sample_prior_control()
            cash=sample_cash(f["sector"],f["branch"],year,digi,age,f["pattern"])
            omzet=sample_jaaromzet(f["size"])
            winst=sample_profit(omzet,f["sector"])
            tax=taxation(winst,f["rechtsvorm"])
            vol_s,vol_l=sample_voluntary(
                year,f["sector"],f["branch"],digi,advisor,cash,
                f["bench"],control,age,f["pattern"]
            )
            pay,correct,timing,risk,hist = risk_and_filing(
                f["sector"],f["size"],f["branch"],digi,advisor,cash,control,vol_s
            )
            anomaly="none"
            if random.random()<P_ANOMALY:
                r=random.random()
                if r<0.33:
                    omzet*=10; winst*=10; tax*=10; anomaly="turnover_outlier"
                elif r<0.66:
                    if vol_l=="hoog":
                        pay="niet_betaald"; correct="foutief_groot"
                        anomaly="high_compliance_but_noncompliant"
                else:
                    if year>=2020 and digi=="hoog":
                        cash="ja"; anomaly="digital_cash_mismatch"
            records.append({
                "bedrijf_id":f["bid"],"jaar":year,
                "pattern_type":f["pattern"],
                "start_jaar":f["start"],"eind_jaar":f["end"],
                "eigenaar_leeftijd":age,
                "eigenaar_geboortejaar":f["birth_year"],
                "eigenaar_geboortedatum":f["birthdate"],
                "grootteklasse":f["size"],"aantal_werknemers":workers,
                "sector_sbi_hoofdcategorie":f["sector"],
                "branche_detail":f["branch"],"regio":f["regio"],
                "plaats":f["plaats"],
                "benchmark_gemeente":"ja" if f["bench"] else "nee",
                "rechtsvorm":f["rechtsvorm"],
                "jaaromzet":omzet,"winst_voor_belasting":winst,
                "verschuldigde_belasting":tax,
                "belasting_betaald_status":pay,
                "aangifte_juisteid":correct,
                "aangifte_tijdigheid":timing,
                "risicocategorie":risk,
                "digitaliseringsniveau":digi,
                "heeft_fiscaal_adviseur":advisor,
                "contant_intensief":cash,
                "eerdere_controle":control,
                "voluntary_compliance_score":round(vol_s,3),
                "voluntary_compliance_level":vol_l,
                "historisch_risico_score":hist,
                "anomaly_flag":anomaly
            })
    return records


# =========================================================
# RUN & SAVE
# =========================================================

if __name__ == "__main__":
    data=generate_panel(N_FIRMS)
    output="synthetic_mkb_belasting_nl_panel_lifecycle.csv"
    fields=list(data[0].keys())
    with open(output,"w",newline="",encoding="utf-8") as f:
        w=csv.DictWriter(f,fieldnames=fields,delimiter=";")
        w.writeheader()
        w.writerows(data)
    print(f"Wrote {len(data)} records → {output}")


Wrote 292830 records → synthetic_mkb_belasting_nl_panel_lifecycle.csv


# Begin modeling mesa ai agents

In [1]:
import pandas as pd

# =========================================================
# LOAD DATA
# =========================================================

df = pd.read_csv("synthetic_mkb_belasting_nl_panel_lifecycle.csv", sep=";")

print("===========================================")
print(" QUICK SUMMARY REPORT – MKB PANEL DATA")
print("===========================================\n")

# =========================================================
# BASIC COUNTS
# =========================================================

n_rows = len(df)
n_firms = df["bedrijf_id"].nunique()
avg_years = n_rows / n_firms

print(f"Total rows (firm-years): {n_rows:,}")
print(f"Total unique firms:      {n_firms:,}")
print(f"Average years per firm:  {avg_years:.2f}\n")


# =========================================================
# PATTERN TYPES
# =========================================================

print("Pattern types (behavioural profiles):")
print(df["pattern_type"].value_counts(normalize=True).rename(lambda x: f"{x}").mul(100).round(2).astype(str) + "%")
print()


# =========================================================
# SECTOR BREAKDOWN
# =========================================================

print("Sector distribution (based on firm-level, unique firms only):")
sector_counts = df.groupby("bedrijf_id")["sector_sbi_hoofdcategorie"].first().value_counts(normalize=True)
print((sector_counts * 100).round(2).astype(str) + "%")
print()


# =========================================================
# DIGITALISATION OVER TIME
# =========================================================

print("Digitalisation trend:")
digi_trend = df.groupby("jaar")["digitaliseringsniveau"].value_counts(normalize=True).unstack().fillna(0)

print(digi_trend.round(3))
print()


# =========================================================
# CASH INTENSITY OVER TIME
# =========================================================

print("Contant-intensief trend:")
cash_trend = df.groupby("jaar")["contant_intensief"].value_counts(normalize=True).unstack().fillna(0)

print(cash_trend.round(3))
print()


# =========================================================
# VOLUNTARY COMPLIANCE OVER TIME
# =========================================================

print("Voluntary compliance trend (average score per year):")
vol_trend = df.groupby("jaar")["voluntary_compliance_score"].mean()

print(vol_trend.round(3))
print()


# =========================================================
# RISK CATEGORIES
# =========================================================

print("Risk category distribution:")
risk_counts = df["risicocategorie"].value_counts(normalize=True)

print((risk_counts * 100).round(2).astype(str) + "%")
print()


# =========================================================
# ANOMALIES
# =========================================================

print("Anomalies report:")
anom = df["anomaly_flag"].value_counts()
total = len(df)
for k, v in anom.items():
    print(f"{k:30s}: {v:,}  ({(v/total)*100:.3f}%)")
print()


# =========================================================
# END
# =========================================================

print("===========================================")
print(" END OF REPORT")
print("===========================================")


 QUICK SUMMARY REPORT – MKB PANEL DATA

Total rows (firm-years): 292,830
Total unique firms:      30,000
Average years per firm:  9.76

Pattern types (behavioural profiles):
pattern_type
normal           96.89%
stuck_in_cash     3.11%
Name: proportion, dtype: object

Sector distribution (based on firm-level, unique firms only):
sector_sbi_hoofdcategorie
Zakelijke diensten              17.73%
Detailhandel                    15.84%
Bouw                            11.49%
Zorg & Welzijn                   8.15%
Horeca                           6.97%
Informatie & ICT                 6.73%
Vervoer & Logistiek              6.22%
Industrie                        5.97%
Persoonlijke dienstverlening     5.08%
Creatieve industrie & Media      5.05%
Financiële dienstverlening        4.1%
Landbouw & Visserij              3.87%
Onderwijs & Overheid              2.8%
Name: proportion, dtype: object

Digitalisation trend:
digitaliseringsniveau   hoog   laag  middel
jaar                                  